# scikit-learn

In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

df = pd.read_csv("../data/Advertising.csv", index_col=0)
df.head()

,TV,Radio,Newspaper,Sales
1,230.1,37.8,69.2,22.1
2,44.5,39.3,45.1,10.4
3,17.2,45.9,69.3,9.3
4,151.5,41.3,58.5,18.5
5,180.8,10.8,58.4,12.9


In [3]:
number_of_samples, number_of_features = df.shape[0], df.shape[1]-1

number_of_samples, number_of_features

(200, 3)

In [4]:
X, y = df.drop("Sales", axis="columns"), df["Sales"]

X.head()

,TV,Radio,Newspaper
1,230.1,37.8,69.2
2,44.5,39.3,45.1
3,17.2,45.9,69.3
4,151.5,41.3,58.5
5,180.8,10.8,58.4


In [7]:
y.head()

1    22.1
2    10.4
3     9.3
4    18.5
5    12.9
Name: Sales, dtype: float64

## Sklearn - typical steps

1. train|test split, sometimes train|val|test split
2. scaling sometimes required
    - min-max scaling
    - standardization
    - ...
    - scale the training data,
    - scale test data to the training data --> avoiding data leakage
3. fit algorithm to training data - model training
4. predict test data
5. evaluate

## Train|test split

In [9]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((140, 3), (60, 3), (140,), (60,))

## Feature saling

Normalization (min-max feature scaling)
- $X' = \frac{X-X_{\min}}{X_{max}-X_{min}}$

 Feature sttandardization
- $X' = \frac{X-\mu}{\sigma}$

In [10]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaler.fit(X_train) # only use on training data not all or test to avoid leakage

scaled_X_train = scaler.transform(X_train)
scaled_X_test = scaler.transform(X_test)

print(f"{scaled_X_train.min()=}")
print(f"{scaled_X_train.max()=}")
print(f"{scaled_X_test.min()=}")
print(f"{scaled_X_test.max()=}")

scaled_X_train.min()=0.0
scaled_X_train.max()=1.0
scaled_X_test.min()=0.005964214711729622
scaled_X_test.max()=1.1302186878727631


In [11]:
scaled_X_train.shape, scaled_X_test.shape

((140, 3), (60, 3))

## Linear regression

### OLS - ordinary least squares

In [13]:
from sklearn.linear_model import LinearRegression

model_OLS = LinearRegression()
model_OLS.fit(scaled_X_train, y_train) # fit on training data

print(f"parameters {model_OLS.coef_}") # beta 1, beta 2, beta 3
print(f"intercept {model_OLS.intercept_}") # beta 0



parameters [13.02832938  9.88465985  0.69237469]
intercept 2.7418553248528124


### SGD - stochastic gradient descent

In [15]:
from sklearn.linear_model import SGDRegressor

model_SGD = SGDRegressor(loss="squared_error",max_iter=1000)
model_SGD.fit(scaled_X_train, y_train)

print(f"parameters {model_SGD.coef_}") # beta 1, beta 2, beta 3
print(f"intercept {model_SGD.intercept_}") # beta 0


parameters [11.93062062  8.98287221  1.33676034]
intercept [3.60495421]


## Manual prediction

In [21]:
test_sample_features = scaled_X_test[0].reshape(1,-1)
test_sample_label = y_test.values[0]
test_sample_features, test_sample_label

(array([[0.54988164, 0.63709677, 0.52286282]]), 16.9)

In [23]:
model_OLS.predict(test_sample_features)[0]

16.56539629743484

In [24]:
model_SGD.predict(test_sample_features)[0]

16.58728459876539

## Evaluation

In [33]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

y_pred_OLS = model_OLS.predict(scaled_X_test)
y_pred_SGD = model_SGD.predict(scaled_X_test)

y_pred_OLS[:5]

array([16.5653963 , 21.18822792, 21.55107058, 10.88923816, 22.20231988])

In [27]:
y_pred_SGD[:5]

array([16.5872846 , 20.79823602, 21.0897182 , 11.32821959, 21.37790813])

In [28]:
y_test[:5].values

array([16.9, 22.4, 21.4,  7.3, 24.7])

In [38]:
mae_OLS = mean_absolute_error(y_test, y_pred_OLS)
mae_SGD = mean_absolute_error(y_test, y_pred_SGD)

mse_OLS = mean_squared_error(y_test, y_pred_OLS)
mse_SGD = mean_squared_error(y_test, y_pred_SGD)

rmse_OLS = np.sqrt(mse_OLS)
rmse_SGD = np.sqrt(mse_SGD)

print(f"{mae_OLS=:.4f} \t {mse_OLS=:.4f} \t {rmse_OLS=:.4f}")
print(f"{mae_SGD=:.4f} \t {mse_SGD=:.4f} \t {rmse_SGD=:.4f}")

mae_OLS=9.3902 	 mse_OLS=108.6873 	 rmse_OLS=10.4253
mae_SGD=9.3902 	 mse_SGD=108.6873 	 rmse_SGD=10.4253
